In [30]:
import pandas as pd

from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.cluster import AgglomerativeClustering

#my utils
from utils import  PkdbModel
from circos_utils import substance_cooccurrence_matrix \
    ,study_expand, bubbles_data, find_label, add_label_and_type, create_config_files, substance_combinations \
    ,create_ideogram, create_links

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration for Circular Plots

This notebook creates the configuration files for the circular plots.

Tasks performed by the notebook:

 1. Classifies substances by co-occurrence in studies. Derived substances are replaced but their parents already by the database. Label for a class is chosen to be most frequent substance within a class.
 2. Classifies studies into the same classes as substances. The most frequent class of substances used in a study is chosen to be the class of the study.
 3. Builds configuration files for circus plots for:
     1. Circular study plot with bars .  &rightarrow;  /circos/study/
     2. Circular study plot with bubbles.   &rightarrow;  /circos/study1/
     3. Circular plots with subset of study plots for each substance class. &rightarrow;  /circos/substance1/
     4. Circular substance plots with co-occurrence links and bubbles. &rightarrow;  /circos/study_caffeine/ ...
     

## 1. Classifies Substances and Studies 

In [32]:
# load data
studies = PkdbModel("studies", destination="1-preprocessed") 
substances = PkdbModel("substances", destination="1-preprocessed") 
substances.read()
studies.read()

In [33]:
coocurence_matrix = substance_cooccurrence_matrix(studies.data, substances.data)

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [34]:
norm_coocurence_matrix = Normalizer(norm="l2").fit_transform(coocurence_matrix)
norm_coocurence_matrix = pd.DataFrame(norm_coocurence_matrix, index=coocurence_matrix.index, columns=coocurence_matrix.columns)

In [35]:
classification_model = AgglomerativeClustering(linkage="single",n_clusters=4)
classes = classification_model.fit_predict(norm_coocurence_matrix.T.corr())

In [36]:
# Classfied substances into types
substance_to_type = pd.DataFrame(classes, index=norm_coocurence_matrix.index, columns=["type"]).to_dict()["type"]

studies.data, substances.data =  add_label_and_type(studies.data, substances.data, substance_to_type)

In [37]:
studies.data

,name,substances,creator,curators,groups_count,group_all_count,groups,individuals_count,individuals,interventions_count,outputs_count,outputs_calculated_count,timecourses_count,results_count,type,label
30,Loetsch2006,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"M K, J G, D E",1,11,"[{'count': 11, 'name': 'all'}]",11,"['P11', 'P10', 'P9', 'P8', 'P7', 'P6', 'P5', '...",3,691,691,132,823,3.0,codeine
130,Arias1988,"5-dehydrosparteine, sparteine sulfate, 2-dehyd...",J G,"J G, M K",1,142,"[{'count': 142, 'name': 'all'}]",142,"['142', '141', '137', '136', '134', '132', '13...",1,568,0,0,568,3.0,codeine
7,Chen1991,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"M K, J G, D E",3,8,"[{'count': 8, 'name': 'all'}, {'count': 1, 'na...",39,"['P_B16', 'P_B15', 'P_B14', 'P_B13', 'P_B12', ...",9,494,28,4,498,3.0,codeine
31,Yue1991B,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"J G, M K",5,20,"[{'count': 20, 'name': 'all'}, {'count': 8, 'n...",16,"['14_PM', '13_PM', '12_PM', '11_PM', '10_PM', ...",11,413,83,11,424,3.0,codeine
107,Liukas2011,"paracetamol sulfate, paracetamol glucuronide, ...",J B,"J B, M K",5,40,"[{'count': 10, 'name': '20-40'}, {'count': 10,...",238,"['Individual 238', 'Individual 237', 'Individu...",1,385,91,12,397,1.0,paracetamol
38,Wu2014,"morphine-3-glucuronide, morphine-6-glucuronide...",J G,"M K, J G, D E",4,29,"[{'count': 29, 'name': 'all'}, {'count': 10, '...",208,"['1_10/10_MOR_AUC', '10_1/10_MOR_AUC', '9_1/10...",1,379,96,12,391,3.0,codeine
13,Thummel1996,"midazolam, 1-hydroxymidazolam",Y D,"Y D, M K",3,20,"[{'count': 20, 'name': 'all'}, {'count': 10, '...",20,"['p20', 'p19', 'p18', 'p17', 'p16', 'p15', 'p1...",2,357,64,8,365,0.0,midazolam
99,Kietzmann1990,"paracetamol, paracetamol sulfate, paracetamol ...",J B,"M K, J B",3,13,"[{'count': 13, 'name': 'all'}, {'count': 7, 'n...",39,"['case26', 'case25', 'case24', 'case23', 'case...",1,320,203,26,346,1.0,paracetamol
23,Vree1992,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"J G, M K",1,8,"[{'count': 8, 'name': 'all'}]",8,"['P8', 'P7', 'P6', 'P5', 'P4', 'P3', 'P2', 'P1']",1,306,40,14,320,3.0,codeine
35,Yue1989-2,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"J G, M K",3,132,"[{'count': 132, 'name': 'all'}, {'count': 18, ...",132,"['132_EM', '131_EM', '130_EM', '129_EM', '128_...",2,301,0,0,301,3.0,codeine


In [38]:
studies_data = studies.data.sort_values(by=["label","name"])

In [39]:
import numpy as np

In [40]:
studies_data["group_all_count"]= studies_data["group_all_count"].replace(np.nan, 0)
studies_data["group_all_count"]= studies_data["group_all_count"].astype(int)

## 2. Build config files for circular plots

In [41]:
# for study data
create_config_files(studies_data, "circos/study_final")

/home/janekg/Dev/pkdb_analysis/circos_utils.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["start"] = range(0,len(data)*rows_per_study,rows_per_study)
/home/janekg/Dev/pkdb_analysis/circos_utils.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["end"] = range(rows_per_study,(len(data)+1)*rows_per_study,rows_per_study)


In [42]:
studies_data

,name,substances,creator,curators,groups_count,group_all_count,groups,individuals_count,individuals,interventions_count,outputs_count,outputs_calculated_count,timecourses_count,results_count,type,label
8,Ammon2002,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"J G, M K, D E",1,12,"[{'count': 12, 'name': 'all'}]",0,NaN,3,153,70,10,163,3.0,codeine
130,Arias1988,"5-dehydrosparteine, sparteine sulfate, 2-dehyd...",J G,"J G, M K",1,142,"[{'count': 142, 'name': 'all'}]",142,"['142', '141', '137', '136', '134', '132', '13...",1,568,0,0,568,3.0,codeine
11,Caraco1996,"mephenytoin, morphine-6-glucuronide, codeine-6...",J G,"J G, M K",3,16,"[{'count': 16, 'name': 'all'}, {'count': 6, 'n...",15,"['6_PM', '5_PM', '4_PM', '3_PM', '2_PM', '1_PM...",4,179,96,12,191,3.0,codeine
7,Chen1991,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"M K, J G, D E",3,8,"[{'count': 8, 'name': 'all'}, {'count': 1, 'na...",39,"['P_B16', 'P_B15', 'P_B14', 'P_B13', 'P_B12', ...",9,494,28,4,498,3.0,codeine
15,Desmeules1991,"dextrorphan, morphine, dextromethorphan, codei...",M K,"M K, D E",2,8,"[{'count': 8, 'name': 'all'}, {'count': 7, 'na...",1,['pm1'],3,29,15,3,32,3.0,codeine
10,Hasselstrom1997,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"J G, M K",3,24,"[{'count': 24, 'name': 'all'}, {'count': 12, '...",0,NaN,2,20,0,0,20,3.0,codeine
20,He2008,"morphine-6-glucuronide, codeine-6-glucuronide,...",J G,"M K, D E, J G",3,23,"[{'count': 23, 'name': 'all'}, {'count': 11, '...",72,"['P_E38', 'P_E37', 'P_E36', 'P_E35', 'P_E34', ...",1,74,0,4,78,3.0,codeine
37,Kim2002,"codeine, codeine sulphate, norcodeine",J G,"J G, M K",5,19,"[{'count': 19, 'name': 'all'}, {'count': 17, '...",0,NaN,2,91,59,8,99,3.0,codeine
17,Kirchheiner2007,"morphine-6-glucuronide, morphine, codeine, cod...",J G,"M K, J G, D E",4,26,"[{'count': 26, 'name': 'all'}, {'count': 3, 'n...",0,NaN,1,96,96,12,108,3.0,codeine
18,Kronstrand2001,"codeine, morphine, codeine phosphate",J G,"M K, D E, J G",1,9,"[{'count': 9, 'name': 'all'}]",0,NaN,1,17,16,3,20,3.0,codeine


In [43]:
for label, study_data_label in studies_data.groupby("label"):
    if label =="paracetamol":
        create_config_files(study_data_label, f"circos/study_{label}")


In [44]:
# testing for substances 

In [45]:
frames = []
for substance, data in substances.data.sort_values("label").groupby("label"):
    data["start"] = range(len(data)) 
    data["end"] = range(1,len(data)+1) 
    frames.append(data)
substances_data = pd.concat(frames)

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [46]:
#substances_data

In [47]:
len(substances_data)

71

In [48]:
unused_substances = substances_data[ (substances_data["study_number"]<2) | ((substances_data["intervention_number"]==0) & (substances_data["timecourse_number"] == 0)  & (substances_data["output_number"] == 0))]["name"]

In [49]:
substances_data = substances_data[ ~((substances_data["study_number"]<2) | ( (substances_data["intervention_number"]==0) & (substances_data["timecourse_number"] == 0)  & (substances_data["output_number"] == 0)))]

In [50]:
len(substances_data)

29

In [51]:
list(unused_substances)

['sulfapyridine',
 'normorphine-glucuronide',
 'dextrorphan',
 '2-dehydrosparteine',
 '5-dehydrosparteine',
 'diclofenac',
 '4-hydroxydebrisoquine',
 'sulfasalazine',
 'mephenytoin',
 'norcodeine-conjugates',
 'ketoconazole',
 'grapefruit juice',
 'atropine',
 'panadol extend',
 'acetylsalicylic acid',
 '4-hydroxyflurbiprofen',
 'nizatidine',
 'imatinib',
 'curcuminoids',
 'flurbiprofen',
 'isoniazide',
 'salicylamide glucuronide',
 'l-cysteine',
 'lignocaine',
 'meperidine',
 'naloxone',
 'cisapride',
 'salicylamide',
 'sulfinpyrazone',
 'probenecid',
 'sulfamethizole acetylated',
 'sorbitol',
 'salicylic acid',
 'phenylbutazone',
 'salicylamide sulfate',
 'propanolol',
 'piperine',
 'pentazocine',
 'sulfamethizole',
 'watercress',
 'pipemidic acid',
 'norfloxacin']

In [52]:
all_links = create_links(substances_data,studies_data,list(unused_substances))


In [53]:
length = pd.DataFrame()
length["substance_links"] = all_links["substance1_start"].append(all_links["substance2_start"])
length["label"] = all_links["substance1_label"].append(all_links["substance2_label"])

In [54]:
frames = []
for label, data_label in length.groupby(["label"]):
    start = 0 
    
    
    for substance_link, substance_linkpositions in data_label.groupby(["substance_links"]):    
        label_position = pd.Series()
        label_position["start"] = start
        label_position["end"] = start + len(substance_linkpositions)
        label_position["label"]  = label
        label_position["substance_number"]  = substance_link

        frames.append(label_position)
        start = label_position["end"] + 1
res = pd.concat(frames, axis=1).T    
                                                         
                                            

In [55]:
substances_data["start"]

63     0
13     2
31     4
17     5
56     7
64     8
16    12
40    15
36    16
14    17
32    18
33    19
38    20
12    21
30    23
11    24
29     0
0      1
43     6
45    19
48    20
44    21
47    22
46    23
6     24
8     25
9     26
54    27
57    29
Name: start, dtype: int64

In [56]:
res["substance_number"] = res["substance_number"].astype(int)

In [57]:
substance_data = pd.merge(res,substances_data[["name","label","start","study_number","timecourse_number","output_number","output_raw_number","output_calculated_number","intervention_number"]],left_on=["label","substance_number"], right_on = ["label","start"])
substance_data["start"] = substance_data["start_x"]

In [58]:
links_new = pd.merge(all_links, substance_data[["start","end","start_y","label"]], left_on=["substance1_label","substance1_start"], right_on = [ "label","start_y"])
links_new[["substance1_start","substance1_end"]] = links_new[["start","end"]]
links_new = pd.merge(links_new, substance_data[["start","end","start_y","label"]], left_on=["substance2_label","substance2_start"], right_on = [ "label","start_y"])
links_new[["substance2_start","substance2_end"]] = links_new[["start_y","end_y"]]


In [59]:
all_links = links_new[["substance1_label","substance1_start" ,"substance1_end", "substance2_label","substance2_start" ,"substance2_end"]]

In [60]:
frames = []
for (label,start), data in all_links.groupby(["substance1_label","substance1_start"]):
    data["substance1_start"] = start + range(len(data))
    data["substance1_end"] = data["substance1_start"] + 1
    data["substance1_label"] = label

    frames.append(data)
    
all_links = pd.concat(frames)

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value i

In [61]:
frames = []
for (label,end), data in all_links.groupby(["substance2_label","substance2_end"]):
    data["substance2_end"] = end - range(len(data)) 
    data["substance2_start"] = data["substance2_end"] - 1
    data["substance2_label"] = label

    frames.append(data)
    
all_links = pd.concat(frames)

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value i

In [62]:
all_links.to_csv("circos/substance_final/data/links.txt", sep=" ",header=False,index=False)

In [63]:
ideogram = pd.DataFrame(res.sort_values("label").groupby(["label"]).apply(lambda x:  x["end"].max()),columns = ["len"])
ideogram["label"] = ideogram.index

In [64]:
ideogram

,len,label
label,,
codeine,1181,codeine
midazolam,19,midazolam
paracetamol,624,paracetamol


In [65]:
ticks_dict = {}
for label, data in substance_data.groupby("label"):
    mystring = ""
    for x in list(data["start"]):
        mystring += f"{x}u, "
    data["end"].max()
    mystring += f"{data['end'].max()}u"
    ticks_dict[label]=mystring

In [66]:
from pprint import pprint
pprint(ticks_dict)

{'codeine': '0u, 20u, 26u, 34u, 51u, 69u, 85u, 142u, 230u, 339u, 457u, 572u, '
            '690u, 772u, 895u, 1037u, 1181u',
 'midazolam': '0u, 11u, 19u',
 'paracetamol': '0u, 147u, 280u, 410u, 494u, 573u, 589u, 602u, 609u, 616u, '
                '619u, 624u'}


In [67]:
directory = "circos/substance_final"
with open(f"{directory}/data/ideogram.txt","w") as f:
        number = 0
        for idx, substance in ideogram.iterrows():

            number = number + 1
            #color_mapping[substance.label] = f"fill_color=spectral-5-div-{number}"
            f.write(f"chr - {substance.label} {substance.label} 0 {substance.len} pastel2-6-qual-{number}\n")


In [68]:
#names 2d track
substance_data["name"] = substance_data["name"].apply(lambda x: x.replace(" ","&nbsp;"))
substance_data[["label","start","end","name"]].to_csv(f"{directory}/data/substance_names.txt", sep=" ",header=False,index=False)
# all subjects number for number track
substance_data[["label","start","end","study_number"]].to_csv(f"{directory}/data/study_number.txt", sep=" ",header=False,index=False)
substance_data[["label","start","end","timecourse_number"]].to_csv(f"{directory}/data/timecourse_number.txt", sep=" ",header=False,index=False)
substance_data[["label","start","end","output_number"]].to_csv(f"{directory}/data/output_number.txt", sep=" ",header=False,index=False)
substance_data[["label","start","end","intervention_number"]].to_csv(f"{directory}/data/intervention_number.txt", sep=" ",header=False,index=False)

bubbles_data_dict = bubbles_data(substance_data,25,"substance")
for name, data in bubbles_data_dict.items():
        data[["label","start","end","type","circle_type"]].to_csv(f"{directory}/data/{name}_bubble.txt", sep=" ",header=False,index=False)


In [ ]:
#old stlye write upper part into utlis !

#create_config_files(substances_data, "circos/substance_final", "substance")
#create_links(substances_data,studies_data).to_csv("circos/substance_final/data/links.txt", sep=" ",header=False,index=False)

In [ ]:
#create_links(substances_data,studies_data)